# Bottle

[Bottle](https://bottlepy.org) ist ein Webframework, das leicht mit pip
installiert werden kann.


## Installation


- Linux, MacOS: `pip3 install bottle`
- Windows: `pip install bottle` oder `python -m pip install bottle` oder `py -m pip install bottle`

## Demo
Das Demoprogramm zeigt eine Webseite mit einem Bild und einer Zahl, die 
sich bei jedem Refresh durch den Browser ändert. 

![Bild](ball.gif)

Für jeden Pfad, der aufgerufen werden kann, muss eine Route in Bottle deklariert
werden. Diese Methode muss einfaches HTML an den Browser liefern. Für Dateien
muss eine separate Route deklariert werden, die Dateien und kein HTML
ausliefert.

In einem [Cheat-Sheet](bottle-cheatsheet.pdf) werden die wichtigsten Dinge
zusammenfasst.

In [1]:
import bottle

Wir beginnen mit der Route für `/`.

In [2]:
@bottle.route("/")  # bindung route to a method
def index():
    html = """
        <!DOCTYPE html>
        <html>
          <head><title>Bottle Demo</title></head>
          <body>
            <h1>Bottle-Demo</h1>
          </body>
        </html>
        """

    return html

Zum Starten des Webservers wird schließlich die Methode `bottle.run` verwendet.
Danach ist die Seite unter http://localhost:8081 erreichbar.
Neben dem Parameter für den `host` und `port` gibt es die Option `reloader=True`.
Sie startet den Server automatisch neu, sobald sich eine der Python-Dateien geändert
hat. Bei der Entwicklung ist dies besonders hilfreich.

In [3]:
bottle.run(host="127.0.0.1", port=8081)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://127.0.0.1:8081/
Hit Ctrl-C to quit.

127.0.0.1 - - [24/Sep/2018 16:14:36] "GET / HTTP/1.1" 200 182


Unter http://127.0.0.1:8081 ist die Seite nun erreichbar.

## Dynamische Webseiten

Nun soll die Seite entwas dynamischer werden und einen Zeitstempel bei jeder Abfrage ausgeben.

In [4]:
import time

@bottle.route("/")  # bindung route to a method
def index():
    t = time.time()
    html = """
        <!DOCTYPE html>
        <html>
          <head><title>Bottle Demo</title></head>
          <body>
            <h1>Bottle-Demo</h1>
            {sekseit1970} Sekunden seit dem 1.1.1970. 
          </body>
        </html>
        """

    # return HTML but replace variable in string before
    return html.format(sekseit1970=t)

Wenn wir den Server nun starten, wird bei jedem Aufruf der Zeitstempel im Ergebnis ersetzt.

In [5]:
bottle.run(host="127.0.0.1", port=8081)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://127.0.0.1:8081/
Hit Ctrl-C to quit.

127.0.0.1 - - [24/Sep/2018 16:14:56] "GET / HTTP/1.1" 200 242
127.0.0.1 - - [24/Sep/2018 16:14:59] "GET / HTTP/1.1" 200 242


## Bilder ausgeben

Schließlich soll noch ein Bild auf der Webseite erscheinen.

In [6]:
@bottle.route("/")  # bindung route to a method
def index():
    html = """
        <!DOCTYPE html>
        <html>
          <head><title>Bottle Demo</title></head>
          <body>
            <h1>Bottle-Demo</h1>
            <img src="/img/ball.gif" />
          </body>
        </html>
        """

    return html

In einem ersten Schritt schlägt die Anzeige jedoch fehl. Es noch eine Route für das Bild.

In [7]:
bottle.run(host="127.0.0.1", port=8081)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://127.0.0.1:8081/
Hit Ctrl-C to quit.

127.0.0.1 - - [24/Sep/2018 16:15:10] "GET / HTTP/1.1" 200 222


In dem HTML-Dokument taucht eine Route `/img/` auf, für die bisher noch 
keine Methode angegeben wurde. Dies holen wir nun nach. Diese Methode 
soll jedoch kein HTML, sondern eine Bilddatei `/img/ball.gif` liefern.

![Bild](ball.gif)

Hierfür können wir die Methode `static_file`im bottle-Modul verwenden.

In [8]:
@bottle.route("/img/<dateiname>")  # dynamic route
def image(dateiname):
    return bottle.static_file(filename=dateiname, root=".")

Wir testen auch diese Methode einmal und rufen sie direkt auf.
Diesmal müssen wir einen Dateiname übergeben, da die Methode
einen Parameter hat. Auch die Route hat einen Parameter.

In [9]:
image("ball.gif")

Content-Type: image/gif
Last-Modified: Sat, 07 Jul 2018 15:57:31 GMT
Content-Length: 5015
Accept-Ranges: bytes

Starten wir den Webserver nun erneut.

In [10]:
bottle.run(host="127.0.0.1", port=8081)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://127.0.0.1:8081/
Hit Ctrl-C to quit.

127.0.0.1 - - [24/Sep/2018 16:16:19] "GET / HTTP/1.1" 200 222


## Formulare

In HTML können mit Formularen Daten an den Webserver übertragen werden. Hierfür wird das [form-tag](https://wiki.selfhtml.org/wiki/HTML/Formulare/form) genutzt. Das `action`-Attribut gibt das Ziel des Formulares an, mit `method` kann die HTTP-Methode für die Übertragung gesetzt werden. Zusätzlich legen wir mit `{req}` einen Teil fest, der später ersetzt werden soll.

In [11]:
@bottle.route("/formular")
def fomular():
    return """
    <!DOCTYPE html>
    <html>
      <head><title>Bottle Demo</title></head>
      <body>
        <h1>Bottle-Formular</h1>

        <form action="/formularauswerter" method="POST">
            <input type="textfield" name="eingabefeld"><br>
            <input type="submit" value="Absenden">
        </form>
        
      </body>
    </html>
    """

Das Formular leitet an die Route `/formularauswerter` weiter. Und zwar mit einem `POST`-Request. Hierfür muss noch eine Methode angegeben werden.

In [12]:
@bottle.post("/formularauswerter")
def fomular():
    html = """
    <!DOCTYPE html>
    <html>
      <head><title>Bottle Demo</title></head>
      <body>
        <h1>Bottle-Formular-Auswerter</h1>
        Es wurde "{req}" eingegeben.
      </body>
    </html>
    """
    formulareingabe = bottle.request.forms["eingabefeld"]

    return html.format(req=formulareingabe)

In [13]:
bottle.run(host="127.0.0.1", port=8081)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://127.0.0.1:8081/
Hit Ctrl-C to quit.

127.0.0.1 - - [24/Sep/2018 16:16:54] "GET /formular HTTP/1.1" 200 348
127.0.0.1 - - [24/Sep/2018 16:16:59] "POST /formularauswerter HTTP/1.1" 200 199


Nun ist das Formular unter http://localhost:8081/formular erreichbar.